In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [2]:
jan_file="https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
feb_file="https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"

In [3]:
jan_df=pd.read_parquet(jan_file)

In [4]:
jan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

In [5]:
jan_df=jan_df.assign(
    duration=lambda d: (d['tpep_dropoff_datetime']-d['tpep_pickup_datetime']).dt.total_seconds()/60
)

In [6]:
round(jan_df.describe().T,2)

,count,mean,std,min,25%,50%,75%,max
VendorID,3066766.0,1.73,0.44,1.00,1.00,2.00,2.00,2.00
passenger_count,2995023.0,1.36,0.90,0.00,1.00,1.00,1.00,9.00
trip_distance,3066766.0,3.85,249.58,0.00,1.06,1.80,3.33,258928.15
RatecodeID,2995023.0,1.50,6.47,1.00,1.00,1.00,1.00,99.00
PULocationID,3066766.0,166.40,64.24,1.00,132.00,162.00,234.00,265.00
DOLocationID,3066766.0,164.39,69.94,1.00,114.00,162.00,234.00,265.00
payment_type,3066766.0,1.19,0.53,0.00,1.00,1.00,1.00,4.00
fare_amount,3066766.0,18.37,17.81,-900.00,8.60,12.80,20.50,1160.10
extra,3066766.0,1.54,1.79,-7.50,0.00,1.00,2.50,12.50
mta_tax,3066766.0,0.49,0.10,-0.50,0.50,0.50,0.50,53.16


## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [7]:
jan_df['duration'].between(1,60).mean()

0.9812202822125979

In [8]:
jan_wo_outliers=jan_df.query("duration.between(1,60)")

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [9]:
categorical = ['PULocationID', 'DOLocationID']
jan_wo_outliers[categorical] = jan_wo_outliers[categorical].astype(str)

/tmp/ipykernel_1752/4125198977.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_wo_outliers[categorical] = jan_wo_outliers[categorical].astype(str)


In [10]:
train_dicts=jan_wo_outliers[categorical].to_dict(orient='records')
dv=DictVectorizer()
X_train=dv.fit_transform(train_dicts)
X_train.shape

(3009173, 515)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [11]:
target = 'duration'
y_train = jan_wo_outliers[target].values

In [12]:
lr=LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [13]:
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

7.649261027919939

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

In [14]:
feb_df=(
    pd.read_parquet(feb_file)
    .assign(
        duration=lambda d: (d['tpep_dropoff_datetime']-d['tpep_pickup_datetime']).dt.total_seconds()/60,
        
    )
    .query("duration.between(1,60)")
)

In [15]:
feb_df[categorical] = feb_df[categorical].astype(str)
X_val=dv.transform(
    feb_df[categorical].to_dict(orient='records')
)
y_val=feb_df[target].values

In [16]:
y_val_pred=lr.predict(X_val)
mean_squared_error(y_val, y_val_pred, squared=False)

7.811832638273232